In [1]:
import os

cwd = os.getcwd()

folder = os.path.dirname(cwd)

folder = folder + '/'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

def ZBL(rij, Zi, Zj):

    const = 1/(4*np.pi*55.26349406e-4)

    a = 0.46850/(Zi**0.23 + Zj**0.23)

    x = rij/a

    phi = 0.18175*np.exp(-3.19980*x) + 0.50986*np.exp(-0.94229*x) + \
          0.28022*np.exp(-0.40290*x) + 0.02817*np.exp(-0.20162*x)
    
    return const*phi*(Zi*Zj/rij)

def LJ(r, epsilon, sigma):
    
    x = sigma/r
    E = 4*epsilon*(x**12 - x**6)
    return E

In [3]:
def fit_He_H(rz, rl):
    
    A = np.array([
            
            [rl**i for i in range(6)],
            [rz**i for i in range(6)],
            [i*rl**(i-1) for i in range(6)],
            [i*rz**(i-1) for i in range(6)],
            [np.clip(i*(i-1)*rl**(i-2), a_min = 0, a_max=np.inf) for i in range(6)],
            [np.clip(i*(i-1)*rz**(i-2), a_min = 0, a_max=np.inf) for i in range(6)],
            
    ])

    l = np.log10( LJ(rl,5.9225e-4, 1.333) )

    z = np.log10( ZBL(rz, 2, 1) )

    h = 1e-5

    dl = np.log10( LJ(rl + h,5.9225e-4, 1.333) ) - np.log10( LJ(rl - h,5.9225e-4, 1.333) )

    dz = np.log10( ZBL(rz + h, 2, 1) ) - np.log10( ZBL(rz - h, 2, 1) )

    d2l = np.log10( LJ(rl + h,5.9225e-4, 1.333) ) - 2*l + np.log10( LJ(rl - h, 5.9225e-4, 1.333) )

    d2z = np.log10( ZBL(rz + h, 2, 1) ) - 2*z + np.log10( ZBL(rz - h , 2, 1) ) 

    dl = 0.5*dl/h

    dz = 0.5*dz/h

    d2l = d2l/h**2

    d2z = d2z/h**2

    b = np.array([l, z, dl, dz, d2l, d2z])

    a = np.linalg.solve(A,b)
    
    return a

In [4]:
def spline_He_He(r):

    data = np.loadtxt('%sPotentials/He-Beck1968_modified.table' % folder,
                      skiprows=7)
    spline = interp1d(data[:,1], data[:,2])
    if r == 0:
        return 0.0
    
    if r < data[:,1].min():
        return ZBL(r, 2, 2)
    elif r > data[:,1].max():
        return 0.0
    else:
        return spline(r)


def spline_W_He(r):

    data = np.loadtxt('%sPotentials/W-He-Juslin.table' % folder, skiprows=7)
    spline = interp1d(data[:,1], data[:,2])

    if r == 0:
        return 0.0
    
    if r < data[:,1].min():
        return ZBL(r, 74, 2)
    elif r > data[:,1].max():
        return 0.0
    else:
        return spline(r)
    
def spline_H_He(r, a, rz, rl):
    
    if r == 0:
        return 0.0
    
    if r < rz:
        return ZBL(r, 2, 1)
        
    elif r > rl:
        return LJ(r,5.9225e-4, 1.333)
    else:
        return 10**(a @ np.array([r**i for i in range(6)]))
    

In [5]:
rz = 0.3
rl = 1

a_He_H = fit_He_H(rz,rl)

In [17]:
with open('%sPotentials/Tungsten_Helium_Hydrogen/WHHe_new.eam.alloy' % folder,'w') as file:

    file.write('# W-H-He potential \n')
    file.write('# \n')
    file.write('# \n')

    file.write('3 W H He \n')
    
    Nrho = 5000
    drho = 0.0020000000
    Nr   = 10000
    dr   = 0.0004850000
    cutoff  = 4.8499999046

    with open('%sPotentials/Tungsten_Hydrogen/MNL6_WH_new.eam.alloy' % folder, 'r') as ref:

        for i in range(4):
            ref.readline()
        
        line = ref.readline()
        file.write(line)
        
        line = ref.readline()
        file.write(line)

        for i in range(Nrho+Nr):

            line = ref.readline()
            file.write(line)

        line = ref.readline()
        file.write(line)

        for i in range(Nrho+Nr):

            line = ref.readline()
            file.write(line)
        
        #The lattice type and constant don't really matter for Helium
        file.write('2  4.0026        10.14484257  DIMER\n')

        for i in range(Nrho+Nr):

            file.write('%16.8f \n' % 0)
        
        for i in range(3*Nr):
            line = ref.readline()
            file.write(line)
        
        for i in range(Nr):

            r = i*dr
            E = r*spline_W_He(r)

            file.write('%16.8f \n' % E)
        
        for i in range(Nr):

            r = i*dr
            E = r*spline_H_He(r, a_He_H, rz, rl)

            file.write('%16.8f \n' % E)

        for i in range(Nr):

            r = i*dr
            E = r*spline_He_He(r)

            file.write('%16.8f \n' % E)
